In [2]:
import os
import math
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta

import sys
print("Python:", sys.version.splitlines()[0])
print("pandas:", pd.__version__)
print("numpy:", np.__version__)

excel_path = r"C:\Users\Dell\Downloads\Supply chain logisitcs problem.xlsx"

out_dir = Path("data/raw")
out_dir.mkdir(parents=True, exist_ok=True)


Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
pandas: 2.3.3
numpy: 2.3.3


- Kiểm tra tên sheets & load cần thiết

In [3]:
!pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]



In [3]:
xls = pd.ExcelFile(excel_path)
print("Danh sách sheet có trong file:", xls.sheet_names)

# Đọc các sheet chính
orderlist = pd.read_excel(excel_path, sheet_name="OrderList")
print("Kích thước OrderList:", orderlist.shape)

freightrates = None
plantports = None

# Nếu có sheet FreightRates → đọc vào
if "FreightRates" in xls.sheet_names:
    freightrates = pd.read_excel(excel_path, sheet_name="FreightRates")
    print("Kích thước FreightRates:", freightrates.shape)
else:
    print("Không tìm thấy sheet FreightRates.")

# Nếu có sheet về cảng (tọa độ) thì đọc vào
if "PlantPorts" in xls.sheet_names:
    plantports = pd.read_excel(excel_path, sheet_name="PlantPorts")
    print("Kích thước PlantPorts:", plantports.shape)
elif "Port_Locations" in xls.sheet_names:
    plantports = pd.read_excel(excel_path, sheet_name="Port_Locations")
    print("Kích thước Port_Locations:", plantports.shape)
else:
    print("Không có sheet chứa tọa độ cảng, sẽ tạm bỏ qua phần tính khoảng cách.")

Danh sách sheet có trong file: ['OrderList', 'FreightRates', 'WhCosts', 'WhCapacities', 'ProductsPerPlant', 'VmiCustomers', 'PlantPorts']
Kích thước OrderList: (9215, 14)
Kích thước FreightRates: (1540, 11)
Kích thước PlantPorts: (22, 2)


- Xem thông tin cột trong OrderList

In [4]:
print("Các cột trong OrderList:")
print(orderlist.columns.tolist())
display(orderlist.head(5))

# Thông tin tổng quan
orderlist.info()


Các cột trong OrderList:
['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT', 'Service Level', 'Ship ahead day count', 'Ship Late Day count', 'Customer', 'Product ID', 'Plant Code', 'Destination Port', 'Unit quantity', 'Weight']


,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.30
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.94
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.20
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.16
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.34


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9215 entries, 0 to 9214
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Order ID              9215 non-null   float64       
 1   Order Date            9215 non-null   datetime64[ns]
 2   Origin Port           9215 non-null   object        
 3   Carrier               9215 non-null   object        
 4   TPT                   9215 non-null   int64         
 5   Service Level         9215 non-null   object        
 6   Ship ahead day count  9215 non-null   int64         
 7   Ship Late Day count   9215 non-null   int64         
 8   Customer              9215 non-null   object        
 9   Product ID            9215 non-null   int64         
 10  Plant Code            9215 non-null   object        
 11  Destination Port      9215 non-null   object        
 12  Unit quantity         9215 non-null   int64         
 13  Weight            

- Chuẩn hóa tên cột

In [5]:
cols_map = {
    "Order ID": "order_id",
    "Order Date": "order_date",
    "Origin Port": "origin_port",
    "Destination Port": "dest_port",
    "Carrier": "carrier",
    "TPT": "tpt_days",
    "Ship ahead day count": "ship_ahead_days",
    "Ship Late Day count": "ship_late_days",
    "Product ID": "product_id",
    "Unit quantity": "unit_quantity",
    "Weight": "weight",
    "Service Level": "service_level",
    "Plant Code": "plant_code"
}

# Đổi tên các cột có trong mapping
mapped = {k:v for k,v in cols_map.items() if k in orderlist.columns}
orderlist = orderlist.rename(columns=mapped)

print("Các cột sau khi đổi tên:")
print(orderlist.columns.tolist())


Các cột sau khi đổi tên:
['order_id', 'order_date', 'origin_port', 'carrier', 'tpt_days', 'service_level', 'ship_ahead_days', 'ship_late_days', 'Customer', 'product_id', 'plant_code', 'dest_port', 'unit_quantity', 'weight']


- Tạo trip_id và tính start_time

In [6]:
# Tạo ID chuyến
orderlist['trip_id'] = orderlist['order_id'].astype(str).str.strip()

# Chuyển định dạng ngày
orderlist['order_date'] = pd.to_datetime(orderlist['order_date'], errors='coerce', utc=True)

# Điền giá trị mặc định cho ship_ahead_days nếu thiếu
if 'ship_ahead_days' not in orderlist.columns:
    orderlist['ship_ahead_days'] = 0
else:
    orderlist['ship_ahead_days'] = pd.to_numeric(orderlist['ship_ahead_days'], errors='coerce').fillna(0)

# Tính thời gian bắt đầu = ngày đặt hàng + số ngày chuẩn bị
orderlist['start_time'] = orderlist['order_date'] + pd.to_timedelta(orderlist['ship_ahead_days'], unit='D')

display(orderlist[['trip_id','order_date','ship_ahead_days','start_time']].head(5))

,trip_id,order_date,ship_ahead_days,start_time
0,1447296446.7,2013-05-26 00:00:00+00:00,3,2013-05-29 00:00:00+00:00
1,1447158014.7,2013-05-26 00:00:00+00:00,3,2013-05-29 00:00:00+00:00
2,1447138898.7,2013-05-26 00:00:00+00:00,3,2013-05-29 00:00:00+00:00
3,1447363527.7,2013-05-26 00:00:00+00:00,3,2013-05-29 00:00:00+00:00
4,1447363980.7,2013-05-26 00:00:00+00:00,3,2013-05-29 00:00:00+00:00


- Ghép bảng FreightRates để bổ sung thời gian vận chuyển (TPT)

In [11]:
if freightrates is not None:
    print("Các cột trong FreightRates:", freightrates.columns.tolist())
    fr = freightrates.copy()

    # Tìm các cột gốc/đích/vận tải/thời gian/giá
    possible_origin = [c for c in fr.columns if 'orig' in c.lower() or 'origin' in c.lower()]
    possible_dest = [c for c in fr.columns if 'dest' in c.lower()]
    possible_carrier = [c for c in fr.columns if 'carrier' in c.lower()]
    possible_tpt = [c for c in fr.columns if 'tpt' in c.lower() or 'day' in c.lower()]
    possible_rate = [c for c in fr.columns if 'rate' in c.lower()]

    print("Xác định được:", possible_origin, possible_dest, possible_carrier, possible_tpt, possible_rate)

    # Chuẩn hóa dữ liệu
    for col in [*possible_origin, *possible_dest, *possible_carrier]:
        fr[col] = fr[col].astype(str).str.strip()

    orderlist['origin_port'] = orderlist['origin_port'].astype(str).str.strip()
    orderlist['dest_port'] = orderlist['dest_port'].astype(str).str.strip()
    orderlist['carrier'] = orderlist['carrier'].astype(str).str.strip()

    # Lựa chọn các cột khóa chung để ghép
    merge_cols = []
    if possible_carrier: merge_cols.append(possible_carrier[0])
    if possible_origin: merge_cols.append(possible_origin[0])
    if possible_dest: merge_cols.append(possible_dest[0])

    if merge_cols:
        # Đổi tên cột trong FreightRates để khớp khi merge
        rename_map = {}
        if possible_carrier: rename_map[possible_carrier[0]] = 'carrier'
        if possible_origin: rename_map[possible_origin[0]] = 'origin_port'
        if possible_dest: rename_map[possible_dest[0]] = 'dest_port'
        fr = fr.rename(columns=rename_map)

        keep_cols = [c for c in [possible_tpt[0] if possible_tpt else None,
                                possible_rate[0] if possible_rate else None] if c]
        fr = fr[['carrier','origin_port','dest_port'] + keep_cols].drop_duplicates()

        merged = orderlist.merge(fr, on=['carrier','origin_port','dest_port'], how='left')
    else:
        merged = orderlist.copy()
else:
    merged = orderlist.copy()

# Chuẩn hóa giá trị thời gian vận chuyển (TPT)
for c in ['tpt_days','tpt_day_cnt','TPT']:
    if c in merged.columns:
        merged[c] = pd.to_numeric(merged[c], errors='coerce')

# Chọn thời gian vận chuyển cuối cùng (ưu tiên OrderList > FreightRates)
merged['tpt_days_final'] = merged.get('tpt_days', np.nan)
if 'tpt_day_cnt' in merged.columns:
    merged['tpt_days_final'] = merged['tpt_days_final'].fillna(merged['tpt_day_cnt'])
merged['tpt_days_final'] = merged['tpt_days_final'].fillna(1.0)
merged['duration_hours'] = merged['tpt_days_final'] * 24

display(merged[['trip_id','carrier','origin_port','dest_port','tpt_days','tpt_days_final','duration_hours']].head(10))


Các cột trong FreightRates: ['Carrier', 'orig_port_cd', 'dest_port_cd', 'minm_wgh_qty', 'max_wgh_qty', 'svc_cd', 'minimum cost', 'rate', 'mode_dsc', 'tpt_day_cnt', 'Carrier type']
Xác định được: ['orig_port_cd'] ['dest_port_cd'] ['Carrier', 'Carrier type'] ['tpt_day_cnt'] ['rate']


,trip_id,carrier,origin_port,dest_port,tpt_days,tpt_days_final,duration_hours
0,1447296446.7,V44_3,PORT09,PORT09,1,1,24
1,1447158014.7,V44_3,PORT09,PORT09,1,1,24
2,1447138898.7,V44_3,PORT09,PORT09,1,1,24
3,1447363527.7,V44_3,PORT09,PORT09,1,1,24
4,1447363980.7,V44_3,PORT09,PORT09,1,1,24
5,1447351440.7,V44_3,PORT09,PORT09,1,1,24
6,1447320235.7,V44_3,PORT09,PORT09,1,1,24
7,1447158018.7,V44_3,PORT09,PORT09,1,1,24
8,1447219340.7,V44_3,PORT09,PORT09,1,1,24
9,1447398415.7,V44_3,PORT09,PORT09,1,1,24


- Tính end_time

In [12]:
if merged['start_time'].isnull().any():
    merged['start_time'] = merged['order_date'].fillna(merged['start_time'])

merged['end_time'] = merged['start_time'] + pd.to_timedelta(merged['duration_hours'], unit='h')

# Nếu thời gian kết thúc nhỏ hơn thời gian bắt đầu → sửa lại
mask_bad = merged['end_time'] < merged['start_time']
if mask_bad.any():
    merged.loc[mask_bad, 'end_time'] = merged.loc[mask_bad, 'start_time'] + pd.Timedelta(hours=1)
    merged.loc[mask_bad, 'duration_hours'] = 1.0

display(merged[['trip_id','start_time','end_time','duration_hours']].head(5))


,trip_id,start_time,end_time,duration_hours
0,1447296446.7,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24
1,1447158014.7,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24
2,1447138898.7,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24
3,1447363527.7,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24
4,1447363980.7,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24


- Map service_level → priority và chọn các cột cần thiết

In [13]:
def map_service_to_priority(s):
    if pd.isna(s): return "Trung bình"
    s = str(s).upper()
    if "EXP" in s or "PRIO" in s:
        return "Cao"
    if "STD" in s:
        return "Trung bình"
    return "Thấp"

merged['priority'] = merged['service_level'].apply(map_service_to_priority)

# Chọn các cột quan trọng cho bảng trip_metadata
trip_meta = merged.rename(columns={
    'origin_port':'origin',
    'dest_port':'destination',
    'unit_quantity':'unit_quantity',
    'weight':'weight',
    'product_id':'product_id'
})[[
    'trip_id','carrier','start_time','end_time','duration_hours',
    'origin','destination','product_id','unit_quantity','weight',
    'service_level','priority','plant_code'
]].copy()

# Thêm cột phụ
trip_meta['expected_delay_min'] = merged.get('ship_late_days', 0).fillna(0) * 24 * 60
trip_meta['target_temp_c'] = 4.0  # nhiệt độ bảo quản mặc định cho hàng tươi

trip_meta.head(6)

,trip_id,carrier,start_time,end_time,duration_hours,origin,destination,product_id,unit_quantity,weight,service_level,priority,plant_code,expected_delay_min,target_temp_c
0,1447296446.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,808,14.30,CRF,Thấp,PLANT16,0,4.0
1,1447158014.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,3188,87.94,CRF,Thấp,PLANT16,0,4.0
2,1447138898.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,2331,61.20,CRF,Thấp,PLANT16,0,4.0
3,1447363527.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,847,16.16,CRF,Thấp,PLANT16,0,4.0
4,1447363980.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,2163,52.34,CRF,Thấp,PLANT16,0,4.0
5,1447351440.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,3332,92.80,CRF,Thấp,PLANT16,0,4.0


- Ghép tọa độ cảng và tính khoảng cách (Haversine)

In [14]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # bán kính Trái Đất (km)
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dlambda/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

if plantports is not None:
    pp = plantports.copy()
    print("Các cột trong sheet tọa độ:", pp.columns.tolist())

    lat_cols = [c for c in pp.columns if 'lat' in c.lower()]
    lon_cols = [c for c in pp.columns if 'lon' in c.lower()]
    name_cols = [c for c in pp.columns if 'port' in c.lower() or 'plant' in c.lower()]

    if name_cols and lat_cols and lon_cols:
        name_col = name_cols[0]; lat_col = lat_cols[0]; lon_col = lon_cols[0]
        pp[name_col] = pp[name_col].astype(str).str.strip()
        port_map = pp.set_index(name_col)[[lat_col, lon_col]].to_dict(orient='index')

        def map_port_coords(port_code):
            if port_code in port_map:
                return port_map[port_code][lat_col], port_map[port_code][lon_col]
            return (np.nan, np.nan)

        trip_meta[['origin_lat','origin_lon']] = trip_meta['origin'].apply(lambda x: pd.Series(map_port_coords(x)))
        trip_meta[['dest_lat','dest_lon']] = trip_meta['destination'].apply(lambda x: pd.Series(map_port_coords(x)))

        trip_meta['distance_km'] = trip_meta.apply(
            lambda r: haversine(r['origin_lat'], r['origin_lon'], r['dest_lat'], r['dest_lon'])
            if pd.notna(r['origin_lat']) and pd.notna(r['dest_lat']) else np.nan, axis=1)
        print("Đã tính được khoảng cách giữa các cảng.")
    else:
        print("Không nhận diện được cột tọa độ, cần kiểm tra lại thủ công.")
else:
    print("Không có sheet tọa độ, bỏ qua phần tính khoảng cách.")

Các cột trong sheet tọa độ: ['Plant Code', 'Port']
Không nhận diện được cột tọa độ, cần kiểm tra lại thủ công.


In [15]:
# Nếu không có tọa độ, ước lượng khoảng cách dựa theo tốc độ trung bình
if 'distance_km' not in trip_meta.columns or trip_meta['distance_km'].isna().all():
    print("Không có dữ liệu tọa độ, sẽ ước lượng khoảng cách dựa theo tốc độ 40 km/h.")
    avg_speed_kmh = 40.0
    trip_meta['distance_km'] = (avg_speed_kmh * trip_meta['duration_hours']).round(1)


Không có dữ liệu tọa độ, sẽ ước lượng khoảng cách dựa theo tốc độ 40 km/h.


- Kiểm tra dữ liệu và thống kê

In [16]:
print("Kích thước bảng trip_metadata:", trip_meta.shape)
print("Số trip_id duy nhất:", trip_meta['trip_id'].nunique())

# Cột nào bị thiếu dữ liệu
missing = trip_meta.isnull().sum().sort_values(ascending=False)
print("Số giá trị thiếu mỗi cột:\n", missing[missing>0])

# Top cảng đi và cảng đến
print("Top cảng đi:")
print(trip_meta['origin'].value_counts().head(10))
print("Top cảng đến:")
print(trip_meta['destination'].value_counts().head(10))

# Mô tả thống kê chung
display(trip_meta.describe(include='all').transpose())


Kích thước bảng trip_metadata: (142908, 16)
Số trip_id duy nhất: 9215
Số giá trị thiếu mỗi cột:
 Series([], dtype: int64)
Top cảng đi:
origin
PORT04    142700
PORT09       207
PORT05         1
Name: count, dtype: int64
Top cảng đến:
destination
PORT09    142908
Name: count, dtype: int64


,count,unique,top,freq,mean,min,25%,50%,75%,max,std
trip_id,142908,9215,1447145318.7,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
carrier,142908,3,V444_1,79686,NaN,NaN,NaN,NaN,NaN,NaN,NaN
start_time,142908,NaN,NaN,NaN,2013-05-28 00:54:51.964060928+00:00,2013-05-26 00:00:00+00:00,2013-05-26 00:00:00+00:00,2013-05-27 00:00:00+00:00,2013-05-31 00:00:00+00:00,2013-06-01 00:00:00+00:00,NaN
end_time,142908,NaN,NaN,NaN,2013-05-29 12:13:32.561927680+00:00,2013-05-26 00:00:00+00:00,2013-05-28 00:00:00+00:00,2013-05-28 00:00:00+00:00,2013-06-01 00:00:00+00:00,2013-06-03 00:00:00+00:00,NaN
duration_hours,142908.0,NaN,NaN,NaN,35.311277,0.0,24.0,24.0,48.0,96.0,12.459841
origin,142908,3,PORT04,142700,NaN,NaN,NaN,NaN,NaN,NaN,NaN
destination,142908,1,PORT09,142908,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_id,142908.0,NaN,NaN,NaN,1681224.638831,1613321.0,1672721.0,1686762.0,1689548.0,1702654.0,13985.54158
unit_quantity,142908.0,NaN,NaN,NaN,2233.805525,235.0,329.0,429.0,871.0,561847.0,13087.177744
weight,142908.0,NaN,NaN,NaN,15.554846,0.0,1.496824,4.567551,13.495736,2338.405126,45.128137


In [17]:
# Lưu ra file CSV và Parquet
csv_out = out_dir / "trip_metadata_from_excel.csv"
parquet_out = out_dir / "trip_metadata_from_excel.parquet"

trip_meta.to_csv(csv_out, index=False)
try:
    trip_meta.to_parquet(parquet_out, index=False)
    print(f"✅ Đã lưu thành công!\n- CSV: {csv_out}\n- Parquet: {parquet_out}")
except Exception as e:
    print("⚠️ Lưu CSV thành công nhưng lỗi khi lưu Parquet (chưa cài pyarrow?).")
    print("Lỗi:", e)


✅ Đã lưu thành công!
- CSV: data\raw\trip_metadata_from_excel.csv
- Parquet: data\raw\trip_metadata_from_excel.parquet


- Tạo bản preview nhỏ

In [18]:
display(trip_meta.head(10))
trip_meta.head(100).to_csv(out_dir/"trip_metadata_preview.csv", index=False)
print("Đã lưu file xem nhanh:", out_dir/"trip_metadata_preview.csv")


,trip_id,carrier,start_time,end_time,duration_hours,origin,destination,product_id,unit_quantity,weight,service_level,priority,plant_code,expected_delay_min,target_temp_c,distance_km
0,1447296446.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,808,14.30,CRF,Thấp,PLANT16,0,4.0,960.0
1,1447158014.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,3188,87.94,CRF,Thấp,PLANT16,0,4.0,960.0
2,1447138898.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,2331,61.20,CRF,Thấp,PLANT16,0,4.0,960.0
3,1447363527.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,847,16.16,CRF,Thấp,PLANT16,0,4.0,960.0
4,1447363980.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,2163,52.34,CRF,Thấp,PLANT16,0,4.0,960.0
5,1447351440.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,3332,92.80,CRF,Thấp,PLANT16,0,4.0,960.0
6,1447320235.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,1782,46.90,CRF,Thấp,PLANT16,0,4.0,960.0
7,1447158018.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,427,2.86,CRF,Thấp,PLANT16,0,4.0,960.0
8,1447219340.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,1291,26.60,CRF,Thấp,PLANT16,0,4.0,960.0
9,1447398415.7,V44_3,2013-05-29 00:00:00+00:00,2013-05-30 00:00:00+00:00,24,PORT09,PORT09,1700106,2294,62.20,CRF,Thấp,PLANT16,0,4.0,960.0


Đã lưu file xem nhanh: data\raw\trip_metadata_preview.csv
